In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
%pip install bs4
%pip install lxml
%pip install pandas
from bs4 import BeautifulSoup

In [ ]:

options = Options()
options.add_argument('--headless')  # optional: runs without opening a browser window
options.add_argument('--disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36")




In [184]:
def convert_to_int(value):
    value = value.strip()
    if value == '--' or not value:
        return 0
    try:
        if 'K' in value:
            return int(float(value.replace('K', '').replace(',', '').strip()) * 1_000)
        elif 'L' in value:
            return int(float(value.replace('L', '').replace(',', '').strip()) * 100_000)
        else:
            return int(value.replace(',', ''))
    except ValueError:
        return 0


print(convert_to_int('--'))

0


In [189]:
import pandas as pd
company_data_list = []
for i in range(1,30):
        companyData = pd.DataFrame()
        companyDetails = soup.find_all('div',class_='companyCardWrapper')

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(f'https://www.ambitionbox.com/list-of-companies?page={i}')

        webpage = driver.page_source  # print part of the HTML
        driver.quit()
        soup = BeautifulSoup(webpage,'lxml')

        for company in companyDetails:
        # Company Name
                companyName = company.find('h2',class_='companyCardWrapper__companyName').text.strip()

                # Company Logo
                logo = company.find('div',class_='companyCardWrapper__companyLogo')
                logoUrl = logo.select_one('img').get_attribute_list('src')[0]

                # Rating
                rating = float(company.find('div',class_='rating_text').select_one('div').text.strip())

                # Industry
                industry = company.find('span',class_='companyCardWrapper__interLinking').text.strip()
                industry = industry.split(sep='|')
                industry = [i.strip() for i in industry]
                industryType = industry[0]
                
                # headquaters
                if(len(industry)>1):
                        head_locations = industry[1].split(sep='+')
                        headquaters = head_locations[0]

                        # location_count
                        loaction_count = int(head_locations[1].split(sep=' ')[0])

                # Rated for 
                highly_rated_for = []
                critically_rated_for = []

                rating_blocks = company.find_all('div', class_='companyCardWrapper__ratingComparisonWrapper')
                # print(len(rating_blocks))
                values = company.find_all('span', class_='companyCardWrapper__ratingValues')
                cheader = company.find('span', class_='companyCardWrapper__ratingHeader--critical')
                hheader = company.find('span', class_='companyCardWrapper__ratingHeader--high')
                if not cheader and hheader:
                        highly_rated_for = [i.strip() for i in values[0].text.strip().split(',')]
                if not hheader and cheader:
                        critically_rated_for = [i.strip() for i in values[0].text.strip().split(',')]
                if hheader and cheader:
                        highly_rated_for = [i.strip() for i in values[0].text.strip().split(',')]
                        critically_rated_for = [i.strip() for i in values[1].text.strip().split(',')]


                # Reviews_count
                count = company.find_all('a',class_='companyCardWrapper__ActionWrapper')

                review_count = count[0].find('span',class_='companyCardWrapper__ActionCount').text
                review_count = convert_to_int(review_count)

                # salaries_count
                salaries_count = count[1].find('span',class_='companyCardWrapper__ActionCount').text
                salaries_count = convert_to_int(salaries_count)

                # interviews_count
                interviews_count = count[2].find('span',class_='companyCardWrapper__ActionCount').text
                interviews_count = convert_to_int(interviews_count)

                # jobs_count
                jobs_count = count[3].find('span',class_='companyCardWrapper__ActionCount').text
                jobs_count = convert_to_int(jobs_count)
                
                company_data_list.append({
                        "Name": companyName,
                        "Logo": logoUrl,
                        "Industry Type": industryType,
                        "Headquarters": headquaters,
                        "No. of Locations": loaction_count,
                        "Highly Rated For": highly_rated_for,
                        "Critically Rated For": critically_rated_for,
                        "Rating": rating,
                        "Review Count": review_count,
                        "Interview Count": interviews_count,
                        "Salaries Count": salaries_count,
                        "Job Count": jobs_count
                        })
        print(f"Page {i} scrapped")


Page 1 scrapped
Page 2 scrapped
Page 3 scrapped
Page 4 scrapped
Page 5 scrapped
Page 6 scrapped
Page 7 scrapped
Page 8 scrapped
Page 9 scrapped
Page 10 scrapped
Page 11 scrapped
Page 12 scrapped
Page 13 scrapped
Page 14 scrapped
Page 15 scrapped
Page 16 scrapped
Page 17 scrapped
Page 18 scrapped
Page 19 scrapped
Page 20 scrapped
Page 21 scrapped
Page 22 scrapped
Page 23 scrapped
Page 24 scrapped
Page 25 scrapped
Page 26 scrapped
Page 27 scrapped
Page 28 scrapped
Page 29 scrapped


In [190]:
companyData = pd.DataFrame(company_data_list)
companyData.head()
companyData.to_csv('companies.csv')